In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Assure que le GPU est bien utilisé
print("f")


f


In [24]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
import torch


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model

# Nom du modèle
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Configurer la quantization en 4-bit
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Charger le modèle avec quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"
)

# Configuration de LoRA
lora_config = LoraConfig(
    r=4,  # Réduction de rang pour économiser la mémoire
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],  # Appliquer LoRA sur les couches d'attention
    bias="none",
    task_type="CAUSAL_LM"
)

# Appliquer LoRA au modèle
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 563,200 || all params: 1,100,611,584 || trainable%: 0.0512


In [5]:
# Charger un dataset depuis un fichier JSON local
dataset = load_dataset("json", data_files=r"C:\Users\arthu\Downloads\data.json")


In [6]:
def format_example(example):
    context = example["context"]
    question = example["question"]
    answer = example["answer"]
    return {
        "text": f"Contexte: {context}\n\nQuestion: {question}\n\nRéponse: {answer}"
    }

# Appliquer la transformation
dataset = dataset.map(format_example)

# Afficher un exemple reformatté
print(dataset["train"][0])

{'context': 'How to use the seq() function to generate values for an empty vector in R?', 'question': "I'm trying to fill an empty vector with values in R using the seq() function. I want to generate values from 1 to 10 with a step of 2. How can I do this?", 'answer': "You can use the seq() function to generate values for an empty vector in R as follows: x <- seq(1, 10, by = 2). The 'by' argument specifies the step size. This will generate the following vector: [1] 1 3 5 7 9. If you want to include 10 in the sequence, you can use the 'length.out' argument: x <- seq(1, 10, by = 2, length.out = 6).", 'text': "Contexte: How to use the seq() function to generate values for an empty vector in R?\n\nQuestion: I'm trying to fill an empty vector with values in R using the seq() function. I want to generate values from 1 to 10 with a step of 2. How can I do this?\n\nRéponse: You can use the seq() function to generate values for an empty vector in R as follows: x <- seq(1, 10, by = 2). The 'by' 

In [2]:
# Vérifie si CUDA est dispo
device = "cuda" if torch.cuda.is_available() else "cpu"

# Déplace le modèle sur le GPU si possible
model.to(device)

# Configurer le collateur de données (Gère le padding automatiquement)
data_collator = DataCollatorForSeq2Seq(tokenizer, padding=True)

# Fonction pour transformer le dataset en un format utilisable par le modèle
def preprocess_function(examples):
    text = f" Contexte:\n{examples['context']}\n\n Question:\n{examples['question']}\n\n Réponse:\n{examples['answer']}"

    # Tokenisation du texte
    tokenized = tokenizer(text, truncation=True, padding="max_length", max_length=2048)

    # Les labels doivent être une copie des input_ids avec -100 pour les tokens à ignorer
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized

# Appliquer le preprocessing au dataset
tokenized_dataset = dataset.map(preprocess_function, remove_columns=dataset["train"].column_names)

# 📌 **ICI que tu dois mettre training_args**
training_args = TrainingArguments(
    output_dir="./tinyllama_lora",
    per_device_train_batch_size=2,  # Ajuste selon la mémoire dispo
    gradient_accumulation_steps=4,  # Simule un batch plus grand
    learning_rate=2e-4,
    num_train_epochs=5,
    save_strategy="epoch",
    logging_dir="./logs",
    fp16=True,  # Active le float16 pour économiser la VRAM
    remove_unused_columns=False
)

# Initialisation de Trainer
trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset["train"].shuffle().select(range(2000)),  # Réduit la taille pour éviter l'overflow mémoire
    args=training_args,
    data_collator=data_collator,
    tokenizer=tokenizer
)

# Lancer l'entraînement
trainer.train()

NameError: name 'torch' is not defined

In [19]:
# Sauvegarde de l'adaptateur LoRA
model.save_pretrained("tinyllama_lora_adapter")


In [ ]:
#----------------------------------------------

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # Optimisation pour GPU
    bnb_4bit_quant_type="nf4",  # Format optimisé
    bnb_4bit_use_double_quant=True
)


base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # Remplace `load_in_4bit`
    device_map="auto"  # Utilise automatiquement le GPU
)


model = PeftModel.from_pretrained(base_model, "tinyllama_lora_adapter")


print("✅ Modèle chargé avec succès !")


✅ Modèle chargé avec succès !


In [4]:
prompt = "How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?"

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

output = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(output[0], skip_special_tokens=True))

How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?
How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?

A: You can use the following code to generate different column selections based on multiple vectors and output the results in an R console:

```r
# Create a data frame with three columns
df <- data.frame(a = c(1, 2, 3), b = c(4, 5, 6), c = c(7, 8, 9))

# Create a vector of column names
colnames <- c('a', 'b', 'c')

# Create a vector of column indices
indices <- c(1, 2, 3)

# Create a loop to generate column selections
for (i in 1:length(indices)) {
  # Create a new column based on the selected column indices
  df <- df


In [6]:
from datasets import load_dataset
import evaluate
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
import math


In [10]:
def generate_text(model, tokenizer, prompt, max_tokens=200):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=max_tokens)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# ---- MÉTRIQUES ---- #

def compute_perplexity(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    
    loss = outputs.loss
    return math.exp(loss.item())

def compute_bleu(reference, generated):
    return sentence_bleu([reference.split()], generated.split())

rouge = evaluate.load("rouge")

def compute_rouge(reference, generated):
    return rouge.compute(predictions=[generated], references=[reference])

def compute_meteor(reference, generated):
    return meteor_score([reference.split()], generated.split())

def distinct_ngrams(text, n=1):
    words = text.split()
    ngrams = set(zip(*[words[i:] for i in range(n)]))
    return len(ngrams) / len(words) if words else 0

Using the latest cached version of the module from C:\Users\arthu\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--rouge\b01e0accf3bd6dd24839b769a5fda24e14995071570870922c71970b3a6ed886 (last modified on Fri Mar 14 11:56:17 2025) since it couldn't be found locally at evaluate-metric--rouge, or remotely on the Hugging Face Hub.


In [17]:
prompt = "How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?"
generated_text = generate_text(model, tokenizer, prompt)
reference_text = "Deep learning is a subfield of machine learning that uses neural networks to learn from data."


In [19]:

print(f"Generated Text: {generated_text}\n")
print(f"Perplexity: {compute_perplexity(model, tokenizer, generated_text)}")
#print(f"BLEU Score: {compute_bleu(reference_text, generated_text)}")
print(f"ROUGE Scores: {compute_rouge(reference_text, generated_text)}")
#print(f"METEOR Score: {compute_meteor(reference_text, generated_text)}")
print(f"Dist-1: {distinct_ngrams(generated_text, 1)}, Dist-2: {distinct_ngrams(generated_text, 2)}")



Generated Text: How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?
How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?

A: You can use the following code to generate different column selections based on multiple vectors and output the results in an R console:

```r
# Create a data frame with three columns
df <- data.frame(a = c(1, 2, 3), b = c(4, 5, 6), c = c(7, 8, 9))

# Create a vector of column names
colnames <- c('a', 'b', 'c')

# Create a vector of column indices
indices <- c(1, 2, 3)

# Create a loop to generate column selections
for (i in 1:length(indices)) {
  # Create a new column based on the selected column indices
  df <- df

Perplexity: 1.8280167003839216
ROUGE Scores: {'rouge1': 0.050314465408805034, 'rouge2': 0.0, 'rougeL': 0.03773584905660377, 'rougeLsum': 0.050314465408805034}
Dist-1: 0.4489795918367347, Dist-2: 0.612244

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")
gpt2_model = AutoModelForCausalLM.from_pretrained("gpt2").to("cuda")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

C:\Users\arthu\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\arthu\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
codegen_tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
codegen_model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-mono").to("cuda")


tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

C:\Users\arthu\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\arthu\.cache\huggingface\hub\models--Salesforce--codegen-350M-mono. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

model.safetensors:   0%|          | 0.00/797M [00:00<?, ?B/s]

In [10]:
import json

# Charger le fichier data.json
with open(r"C:\Users\arthu\Downloads\data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Extraire les prompts et les réponses de référence
prompts = [entry["question"] for entry in data]
references = [entry["answer"] for entry in data]


In [14]:
import json
import torch
from transformers import AutoTokenizer
from evaluate import load
from tqdm import tqdm

# --- Charger le fichier data.json ---
with open(r"C:\Users\arthu\Downloads\data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

prompts = [entry["question"] for entry in data]
references = [entry["answer"] for entry in data]

# --- Générer les réponses avec ton modèle TinyLlama ---
def generate_responses(model, tokenizer, prompts):
    generations = []
    model.eval()
    for prompt in tqdm(prompts, desc="🔁 Génération"):
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            output = model.generate(**inputs, max_new_tokens=300)
        decoded = tokenizer.decode(output[0], skip_special_tokens=True)
        generations.append(decoded)
    return generations

tinyllama_outputs = generate_responses(model, tokenizer, prompts)

# --- Calcul des métriques ---
rouge = load("rouge")
bertscore = load("bertscore")

# ROUGE
rouge_result = rouge.compute(predictions=tinyllama_outputs, references=references)
print("\n📊 ROUGE:", rouge_result)

# BERTScore
bert_result = bertscore.compute(predictions=tinyllama_outputs, references=references, lang="en")
print("\n📊 BERTScore (F1 moyenne):", sum(bert_result["f1"]) / len(bert_result["f1"]))



NameError: name 'tokenizer' is not defined

In [4]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from evaluate import load
from tqdm import tqdm

# Charger les réponses de référence
with open(r"C:\Users\arthu\Downloads\data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

references = [entry["answer"] for entry in data]

# Saisir un prompt dans la console
user_prompt = ("How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?")

# Fonction de génération
def generate_text(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=300)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Fonction pour calculer la diversité
def distinct_n(text, n):
    tokens = text.split()
    ngrams = list(zip(*[tokens[i:] for i in range(n)]))
    return len(set(ngrams)) / len(ngrams) if len(ngrams) > 0 else 0

# Initialiser les métriques
rouge = load("rouge")
bertscore = load("bertscore")

# Dictionnaire de modèles à comparer
models_info = {}

# === Ton modèle TinyLlama avec LoRA ===
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)
tokenizer_llama = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
model_lora = PeftModel.from_pretrained(base_model, "tinyllama_lora_adapter")
models_info["TinyLlama+LoRA"] = (model_lora, tokenizer_llama)

# === GPT-2 Baseline ===
tokenizer_gpt2 = AutoTokenizer.from_pretrained("gpt2")
model_gpt2 = AutoModelForCausalLM.from_pretrained("gpt2").to("cuda")
models_info["GPT-2"] = (model_gpt2, tokenizer_gpt2)


# === 2. CodeGen 350M ===
tokenizer_codegen = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
model_codegen = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-mono").to("cuda")
models_info["CodeGen-350M"] = (model_codegen, tokenizer_codegen)


# === Comparaison ===
results = {}

for model_name, (model, tokenizer) in models_info.items():
    print(f"\n⚙️ Génération avec : {model_name}")
    generation = generate_text(model, tokenizer, user_prompt)
    print(f"\n🧠 Réponse générée par {model_name} :\n{generation}\n")

    # Calcul des scores
    rouge_scores = rouge.compute(predictions=[generation] * len(references), references=references)
    bert_result = bertscore.compute(predictions=[generation] * len(references), references=references, lang="en")
    bert_avg = sum(bert_result["f1"]) / len(bert_result["f1"])

    dist1 = distinct_n(generation, 1)
    dist2 = distinct_n(generation, 2)

    results[model_name] = {
        "ROUGE-1": rouge_scores["rouge1"],
        "ROUGE-2": rouge_scores["rouge2"],
        "ROUGE-L": rouge_scores["rougeL"],
        "BERTScore (F1)": bert_avg,
        "Dist-1": dist1,
        "Dist-2": dist2
    }

# === Résumé final ===
import pandas as pd

df = pd.DataFrame(results).T
print("\n📊 Résumé des performances :\n")
print(df.round(4))


Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e


⚙️ Génération avec : TinyLlama+LoRA

🧠 Réponse générée par TinyLlama+LoRA :
How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?
How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?

A: You can use the following code to generate different column selections based on multiple vectors and output the results in an R console:

```r
# Create a data frame with three columns
df <- data.frame(a = c(1, 2, 3), b = c(4, 5, 6), c = c(7, 8, 9))

# Create a vector of column names
colnames <- c('a', 'b', 'c')

# Create a vector of column indices
indices <- c(1, 2, 3)

# Create a loop to generate column selections
for (i in 1:length(indices)) {
  # Create a new column based on the selected column indices
  df <- df[indices[i], ]
  
  # Print the results
  print(df)
}
```

This code will generate three column selections based on the selected column indices.

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



⚙️ Génération avec : GPT-2

🧠 Réponse générée par GPT-2 :
How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?

You can use a loop to generate different column selections based on multiple vectors and output the results in an R console.

You can use a loop to generate different column selections based on multiple vectors and output the results in an R console.

You can use a loop to generate different column selections based on multiple vectors and output the results in an R console.

You can use a loop to generate different column selections based on multiple vectors and output the results in an R console.

You can use a loop to generate different column selections based on multiple vectors and output the results in an R console.

You can use a loop to generate different column selections based on multiple vectors and output the results in an R console.

You can use a loop to generate different column selection

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



⚙️ Génération avec : CodeGen-350M

🧠 Réponse générée par CodeGen-350M :
How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?

# In[ ]:


# Create a list of the values in the x and y axis
x = [1, 2, 3, 4, 5]
y = [1, 2, 3, 4, 5]

# Create a list of the values in the x and y axis
x = [1, 2, 3, 4, 5]
y = [1, 2, 3, 4, 5]

# Create a list of the values in the x and y axis
x = [1, 2, 3, 4, 5]
y = [1, 2, 3, 4, 5]

# Create a list of the values in the x and y axis
x = [1, 2, 3, 4, 5]
y = [1, 2, 3, 4, 5]

# Create a list of the values in the x and y axis
x = [1, 2, 3, 4, 5]
y = [1, 2, 3, 4, 5]

# Create a list of the values in the x and y axis
x = [1, 2, 3, 4, 5]
y = [1, 2, 3, 4, 5]

# Create a list of the values in the x and y axis
x = [1, 2, 3, 4, 5]
y = [1,


📊 Résumé des performances :

                ROUGE-1  ROUGE-2  ROUGE-L  BERTScore (F1)  Dist-1  Dist-2
TinyLlama+LoRA   0.2317   0.0506   0.1454          0.8235  

In [ ]:
import json
import torch
import pandas as pd
from tqdm import tqdm
from evaluate import load
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from peft import PeftModel

# ========== CONFIG ==========
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
lora_adapter_path = "tinyllama_lora_adapter"  # <- ton modèle fine-tuné
data_path = r"C:\Users\arthu\Downloads\data.json"
prompt = "How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?"
device = "cuda" if torch.cuda.is_available() else "cpu"

# ========== MÉTRIQUES ==========
rouge = load("rouge")
bleu = load("bleu")
bertscore = load("bertscore")

def distinct_n(text, n):
    tokens = text.split()
    ngrams = list(zip(*[tokens[i:] for i in range(n)]))
    return len(set(ngrams)) / len(ngrams) if ngrams else 0

# ========== CHARGER LES RÉFÉRENCES ==========
with open(data_path, "r", encoding="utf-8") as f:
    data = json.load(f)

references = [entry["answer"] for entry in data]

# ========== GÉNÉRATION ==========
def generate_text(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=300)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# ========== CHARGER LES MODÈLES ==========
models_info = {}

# --- CONFIG 4bit pour TinyLlama ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# --- 0. TinyLlama (base) ---
tokenizer_base = AutoTokenizer.from_pretrained(model_name)
model_base = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
models_info["TinyLlama (base)"] = (model_base, tokenizer_base)

# --- 1. TinyLlama + LoRA ---
tokenizer_lora = AutoTokenizer.from_pretrained(model_name)
base_model_lora = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
model_lora = PeftModel.from_pretrained(base_model_lora, lora_adapter_path)
models_info["TinyLlama+LoRA"] = (model_lora, tokenizer_lora)

# ========== ÉVALUATION ==========
results = {}

for model_label, (model, tokenizer) in models_info.items():
    print(f"\n🔄 Génération avec {model_label}")
    generation = generate_text(model, tokenizer, prompt)
    print(f"\n🧠 Réponse générée par {model_label} :\n{generation}\n")

    # Évaluation
    rouge_scores = rouge.compute(predictions=[generation] * len(references), references=references)
    bleu_score = bleu.compute(predictions=[generation], references=[[ref] for ref in references])
    bert_result = bertscore.compute(predictions=[generation] * len(references), references=references, lang="en")
    bert_avg = sum(bert_result["f1"]) / len(bert_result["f1"])

    results[model_label] = {
        "ROUGE-1": rouge_scores["rouge1"],
        "ROUGE-2": rouge_scores["rouge2"],
        "ROUGE-L": rouge_scores["rougeL"],
        "BLEU": bleu_score["bleu"],
        "BERTScore (F1)": bert_avg,
        "Dist-1": distinct_n(generation, 1),
        "Dist-2": distinct_n(generation, 2)
    }

# ========== RÉSULTAT FINAL ==========
df = pd.DataFrame(results).T
print("\n📊 Résumé des performances :\n")
print(df.round(4))

# Optionnel : export CSV
# df.to_csv("resultats_modeles.csv")


In [ ]:
import json
import torch
import pandas as pd
from evaluate import load
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from peft import PeftModel

# ========== CONFIGURATION ==========
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
lora_adapter_path = "tinyllama_lora_adapter"  # <- Ton LoRA fine-tuné
data_path = r"C:\Users\arthu\Downloads\data.json"
prompt = "How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?"
device = "cuda" if torch.cuda.is_available() else "cpu"

# ========== MÉTRIQUES UTILES ==========
bleu = load("bleu")
bertscore = load("bertscore")

# Exact match
def exact_match(pred, ref):
    return int(pred.strip().lower() == ref.strip().lower())

# BLEU Accuracy (seuil)
def bleu_accuracy(preds, refs, threshold=0.3):
    correct = 0
    for pred, ref in zip(preds, refs):
        score = bleu.compute(predictions=[pred], references=[[ref]])["bleu"]
        if score >= threshold:
            correct += 1
    return correct / len(preds)

# BERTScore Accuracy (seuil)
def bertscore_accuracy(preds, refs, threshold=0.85):
    scores = bertscore.compute(predictions=preds, references=refs, lang="en")["f1"]
    return sum(s >= threshold for s in scores) / len(scores)

# Exact Match Accuracy global
def em_accuracy(preds, refs):
    return sum([exact_match(p, r) for p, r in zip(preds, refs)]) / len(refs)

# ========== CHARGER LES RÉFÉRENCES ==========
with open(data_path, "r", encoding="utf-8") as f:
    data = json.load(f)

references = [entry["answer"] for entry in data]

# ========== GÉNÉRATION ==========
def generate_text(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=300)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# ========== CHARGER LES MODÈLES ==========
models_info = {}

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    llm_int8_enable_fp32_cpu_offload=True  
)

# --- 0. TinyLlama (base) ---
tokenizer_base = AutoTokenizer.from_pretrained(model_name)
model_base = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
models_info["TinyLlama (base)"] = (model_base, tokenizer_base)

# --- 1. TinyLlama + LoRA ---
tokenizer_lora = AutoTokenizer.from_pretrained(model_name)
base_model_lora = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
model_lora = PeftModel.from_pretrained(base_model_lora, lora_adapter_path)
models_info["TinyLlama+LoRA"] = (model_lora, tokenizer_lora)

# ========== ÉVALUATION ==========
all_results = {}

for model_label, (model, tokenizer) in models_info.items():
    print(f"\n🔄 Génération avec {model_label}")
    generation = generate_text(model, tokenizer, prompt)
    print(f"\n🧠 Réponse générée :\n{generation}\n")

    predictions = [generation] * len(references)

    # Calcul des métriques d'accuracy
    em = em_accuracy(predictions, references)
    bleu_acc = bleu_accuracy(predictions, references)
    bert_acc = bertscore_accuracy(predictions, references)

    all_results[model_label] = {
        "Exact Match Acc": em,
        "BLEU Acc": bleu_acc,
        "BERTScore Acc": bert_acc
    }

# ========== AFFICHAGE FINAL ==========
df = pd.DataFrame(all_results).T
print("\n📊 Résumé des ACCURACIES :\n")
print(df.round(4))



🔄 Génération avec TinyLlama (base)

🧠 Réponse générée :
How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
import json
import torch
import pandas as pd
from evaluate import load
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from peft import PeftModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ========== CONFIGURATION ==========
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
lora_adapter_path = "tinyllama_lora_adapter"  # <- Ton LoRA fine-tuné
data_path = r"C:\Users\arthu\Downloads\data.json"
prompt = "How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?"
device = "cuda" if torch.cuda.is_available() else "cpu"

# ========== MÉTRIQUES UTILES ==========
bleu = load("bleu")
bertscore = load("bertscore")

# Exact match
def exact_match(pred, ref):
    return int(pred.strip().lower() == ref.strip().lower())

# BLEU Accuracy (seuil)
def bleu_accuracy(preds, refs, threshold=0.3):
    correct = 0
    binary_preds = []
    for pred, ref in zip(preds, refs):
        score = bleu.compute(predictions=[pred], references=[[ref]])["bleu"]
        is_correct = score >= threshold
        binary_preds.append(int(is_correct))
        if is_correct:
            correct += 1
    return correct / len(preds), binary_preds

# BERTScore Accuracy (seuil)
def bertscore_accuracy(preds, refs, threshold=0.85):
    scores = bertscore.compute(predictions=preds, references=refs, lang="en")["f1"]
    return sum(s >= threshold for s in scores) / len(scores)

# Exact Match Accuracy global
def em_accuracy(preds, refs):
    return sum([exact_match(p, r) for p, r in zip(preds, refs)]) / len(refs)

# ========== CHARGER LES RÉFÉRENCES ==========
with open(data_path, "r", encoding="utf-8") as f:
    data = json.load(f)

references = [entry["answer"] for entry in data]

# ========== GÉNÉRATION ==========
def generate_text(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=300)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# ========== CHARGER LES MODÈLES ==========
models_info = {}

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# --- 0. TinyLlama (base) ---
tokenizer_base = AutoTokenizer.from_pretrained(model_name)
model_base = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
models_info["TinyLlama (base)"] = (model_base, tokenizer_base)

# --- 1. TinyLlama + LoRA ---
tokenizer_lora = AutoTokenizer.from_pretrained(model_name)
base_model_lora = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
model_lora = PeftModel.from_pretrained(base_model_lora, lora_adapter_path)
models_info["TinyLlama+LoRA"] = (model_lora, tokenizer_lora)

# ========== ÉVALUATION ==========
all_results = {}

for model_label, (model, tokenizer) in models_info.items():
    print(f"\n🔄 Génération avec {model_label}")
    generation = generate_text(model, tokenizer, prompt)
    print(f"\n🧠 Réponse générée :\n{generation}\n")

    predictions = [generation] * len(references)

    # Calcul des métriques
    em = em_accuracy(predictions, references)
    bleu_acc, binary_preds = bleu_accuracy(predictions, references)
    bert_acc = bertscore_accuracy(predictions, references)

    # Classification-based scores
    y_true = [1] * len(references)  # on suppose que toutes les réponses attendues sont positives
    acc = accuracy_score(y_true, binary_preds)
    precision = precision_score(y_true, binary_preds)
    recall = recall_score(y_true, binary_preds)
    f1 = f1_score(y_true, binary_preds)

    all_results[model_label] = {
        "Exact Match Acc": em,
        "BLEU Acc": bleu_acc,
        "BERTScore Acc": bert_acc,
        "Accuracy": acc,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    }

# ========== AFFICHAGE FINAL ==========
df = pd.DataFrame(all_results).T
print("\n📊 Résumé des performances :\n")
print(df.round(4))



🔄 Génération avec TinyLlama (base)

🧠 Réponse générée :
How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\arthu\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🔄 Génération avec TinyLlama+LoRA

🧠 Réponse générée :
How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?
How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?

A: You can use the following code to generate different column selections based on multiple vectors and output the results in an R console:

```r
# Create a data frame with three columns
df <- data.frame(a = c(1, 2, 3), b = c(4, 5, 6), c = c(7, 8, 9))

# Create a vector of column names
colnames <- c('a', 'b', 'c')

# Create a vector of column indices
indices <- c(1, 2, 3)

# Create a loop to generate column selections
for (i in 1:length(indices)) {
  # Create a new column based on the selected column indices
  df <- df[indices[i], ]
  
  # Print the results
  print(df)
}
```

This code will generate three column selections based on the selected column indices. The loop will iterate

C:\Users\arthu\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [2]:
import json
import torch
import pandas as pd
from evaluate import load
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from peft import PeftModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ========== CONFIGURATION ==========
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
lora_adapter_path = "tinyllama_lora_adapter"  # <- Ton LoRA fine-tuné
data_path = r"C:\Users\arthu\Downloads\data.json"
prompt = "How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?"
device = "cuda" if torch.cuda.is_available() else "cpu"

# ========== MÉTRIQUES UTILES ==========
bertscore = load("bertscore")

# BERTScore Accuracy (seuil)
def bertscore_accuracy(preds, refs, threshold=0.85):
    scores = bertscore.compute(predictions=preds, references=refs, lang="en")["f1"]
    binary_preds = [int(s >= threshold) for s in scores]
    return sum(binary_preds) / len(scores), binary_preds

# ========== CHARGER LES RÉFÉRENCES ==========
with open(data_path, "r", encoding="utf-8") as f:
    data = json.load(f)

references = [entry["answer"] for entry in data]

# ========== GÉNÉRATION ==========
def generate_text(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=300)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# ========== CHARGER LES MODÈLES ==========
models_info = {}

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# --- 0. TinyLlama (base) ---
tokenizer_base = AutoTokenizer.from_pretrained(model_name)
model_base = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
models_info["TinyLlama (base)"] = (model_base, tokenizer_base)

# --- 1. TinyLlama + LoRA ---
tokenizer_lora = AutoTokenizer.from_pretrained(model_name)
base_model_lora = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
model_lora = PeftModel.from_pretrained(base_model_lora, lora_adapter_path)
models_info["TinyLlama+LoRA"] = (model_lora, tokenizer_lora)

# ========== ÉVALUATION ==========
all_results = {}

for model_label, (model, tokenizer) in models_info.items():
    print(f"\n🔄 Génération avec {model_label}")
    generation = generate_text(model, tokenizer, prompt)
    print(f"\n🧠 Réponse générée :\n{generation}\n")

    predictions = [generation] * len(references)

    # BERTScore + métriques de classification
    bert_acc, binary_preds = bertscore_accuracy(predictions, references)
    y_true = [1] * len(references)  # Toutes les réponses attendues sont considérées comme positives

    acc = accuracy_score(y_true, binary_preds)
    precision = precision_score(y_true, binary_preds, zero_division=0)
    recall = recall_score(y_true, binary_preds, zero_division=0)
    f1 = f1_score(y_true, binary_preds, zero_division=0)

    all_results[model_label] = {
        "BERTScore Acc": bert_acc,
        "Accuracy": acc,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    }

# ========== AFFICHAGE FINAL ==========
df = pd.DataFrame(all_results).T
print("\n📊 Résumé des performances :\n")
print(df.round(4))



🔄 Génération avec TinyLlama (base)

🧠 Réponse générée :
How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🔄 Génération avec TinyLlama+LoRA

🧠 Réponse générée :
How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?
How can I use a loop to generate different column selections based on multiple vectors and output the results in an R console?

A: You can use the following code to generate different column selections based on multiple vectors and output the results in an R console:

```r
# Create a data frame with three columns
df <- data.frame(a = c(1, 2, 3), b = c(4, 5, 6), c = c(7, 8, 9))

# Create a vector of column names
colnames <- c('a', 'b', 'c')

# Create a vector of column indices
indices <- c(1, 2, 3)

# Create a loop to generate column selections
for (i in 1:length(indices)) {
  # Create a new column based on the selected column indices
  df <- df[indices[i], ]
  
  # Print the results
  print(df)
}
```

This code will generate three column selections based on the selected column indices. The loop will iterate